In [7]:
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.express as px

# Load the weather data into a Pandas DataFrame
weather_data = pd.read_csv('./data/stats_offshore.csv', index_col = 0, parse_dates = True)  # Replace 'weather_data.csv' with your actual data file
weather_data['Date'] = weather_data.index
# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(
    children=[
        html.H1("Weather Data Performance Dashboard"),

        # Dropdown for selecting columns
        dcc.Dropdown(
            id="column-dropdown",
            options=[{'label': col, 'value': col} for col in weather_data.columns],
            value=['hs'],  # Default columns to display
            multi=True,  # Enable multiple column selection
        ),

        # Radio buttons for sorting options
        dcc.RadioItems(
            id="sort-radio",
            options=[
                {'label': 'Ascending', 'value': 'asc'},
                {'label': 'Descending', 'value': 'desc'}
            ],
            value='asc',  # Default sort order
        ),

        # Timeseries plot
        dcc.Graph(id="timeseries-plot"),

        # Distribution plot
        dcc.Graph(id="distribution-plot"),
    ]
)


@app.callback(
    dash.dependencies.Output("timeseries-plot", "figure"),
    dash.dependencies.Output("distribution-plot", "figure"),
    [dash.dependencies.Input("column-dropdown", "value"),
     dash.dependencies.Input("sort-radio", "value")]
)
def update_plots(selected_columns, sort_order):
    # Sort the weather data based on selected column and sort order
    sorted_data = weather_data#.sort_values(selected_columns[0], ascending=(sort_order == 'asc'))

    # Create timeseries plot for sorted data
    timeseries_plot = px.line(sorted_data, x="Date", y=selected_columns, title="Timeseries Comparison")

    # Create distribution plot for selected columns
    distribution_plot = px.histogram(sorted_data, x=selected_columns, title="Distribution Comparison")

    return timeseries_plot, distribution_plot


# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

In [31]:
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the data
df = pd.read_csv('./data/stats_offshore.csv', index_col = 0, parse_dates = True)



# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(
    children=[
        html.H1("Timeseries Data Dashboard"),
        html.Div(
            children=[
                html.Label("Select columns to plot:"),
                dcc.Dropdown(
                    id="columns-dropdown",
                    options=[{"label": col, "value": col} for col in df.columns],
                    value=[df.columns[0]],
                    multi=True,
                ),
            ]
        ),
        html.Div(
            className="row",
            children=[
                html.Div(
                    className="six columns",
                    children=[
                        dcc.Graph(id="timeseries-plot"),
                    ],
                ),
                html.Div(
                    className="six columns",
                    children=[
                        html.Label("Select x-axis column:"),
                        dcc.Dropdown(
                            id="scatter-x-axis",
                            options=[{"label": col, "value": col} for col in df.columns],
                            value=df.columns[0],
                        ),
                        html.Label("Select y-axis column:"),
                        dcc.Dropdown(
                            id="scatter-y-axis",
                            options=[{"label": col, "value": col} for col in df.columns],
                            value=df.columns[1],
                        ),
                        dcc.Graph(id="scatter-plot"),
                    ],
                ),
            ],
        ),
        html.Div(
            className="row",
            children=[
                html.Div(
                    className="six columns",
                    children=[
                        dcc.Graph(id="distribution-plot"),
                    ],
                ),
                html.Div(
                    className="six columns",
                    children=[
                        html.H3("Summary Statistics"),
                        dcc.Graph(id="summary-table"),
                    ],
                ),
            ],
        ),
    ]
)


@app.callback(
    dash.dependencies.Output("timeseries-plot", "figure"),
    dash.dependencies.Output("scatter-plot", "figure"),
    dash.dependencies.Output("distribution-plot", "figure"),
    dash.dependencies.Output("summary-table", "figure"),
    [
        dash.dependencies.Input("columns-dropdown", "value"),
        dash.dependencies.Input("scatter-x-axis", "value"),
        dash.dependencies.Input("scatter-y-axis", "value"),
    ],
)
def update_plots(selected_columns, scatter_x_axis, scatter_y_axis):
    figures = []

    for column in selected_columns:
        # Timeseries plot
        timeseries_fig = px.line(df, x=df.index, y=column, title=f"{column} Timeseries")
        figures.append(timeseries_fig)

        # Scatter plot
        scatter_fig = px.scatter(df, x=scatter_x_axis, y=scatter_y_axis, title="Scatter Plot")
        figures.append(scatter_fig)

        # Distribution plot
        distribution_fig = px.histogram(df, x=column, title=f"{column} Distribution")
        figures.append(distribution_fig)

    # Summary statistics
    stats_data = []
    for column in selected_columns:
        column_stats = df[column].describe().reset_index()
        stats_data.append(column_stats)

    stats_table = go.Table(
        header=dict(values=["Statistic", *selected_columns]),
        cells=dict(values=[stats_data[i].columns[1:] for i in range(len(selected_columns))]),
    )
    summary_stats_fig = go.Figure(data=[stats_table])

    figures.append(summary_stats_fig)

    return figures[0], figures[1], figures[2], figures[3]


# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)